# 📚 Kapitola 32: Ollama API integrace

<div style="background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h2 style="color: white; margin: 0;">Blok 4 | EXPERT</h2>
    <p style="color: white; margin: 10px 0;">📖 Výuková kapitola</p>
</div>

## 🎯 Co se naučíte

V této kapitole se zaměříme na následující témata:

- **Ollama REST API endpoints**
- **Python client implementation**
- **Streaming responses**
- **Model parameters tuning**
- **Context window management**
- **Token counting**
- **Error handling strategies**

## ⚠️ Předpoklady
Tato kapitola navazuje na kapitoly: 31, 13

---


In [ ]:
# ⚙️ Inicializace prostředí
# Tento kód nastavuje prostředí pro kapitolu 32

import sys
import os
from datetime import datetime

# Informace o prostředí
print("=" * 60)
print(f"📚 KAPITOLA 32: Ollama API integrace")
print("=" * 60)
print(f"🐍 Python verze: {sys.version}")
print(f"📅 Datum spuštění: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💻 OS: {os.name}")
print(f"📁 Pracovní adresář: {os.getcwd()}")
print("=" * 60)

# Instalace potřebných knihoven (odkomentujte podle potřeby)
# !pip install requests pandas numpy matplotlib
# !pip install beautifulsoup4 sqlalchemy fastapi

# Import základních knihoven
import json
import csv
import time
import random
from pathlib import Path
from typing import List, Dict, Optional, Any

print("✅ Prostředí připraveno!")

## 📖 Teoretická část

<div style="background: #f0f4ff; padding: 20px; border-left: 5px solid #4a69bd; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🎓 Základní teorie a koncepty</h3>
</div>

# Ollama API Integrace

## 1. Úvod a motivace

V oblasti umělé inteligence se stále více rozvíjí možnost lokálního provozu modelů AI přímo na uživatelském zařízení. Ollama je platforma, která umožňuje jednoduše spouštět a spravovat různé LLM (Large Language Models) lokálně, včetně možnosti jejich integrace přes REST API. Význam této kapitoly spočívá v tom, že studenti se naučí, jak efektivně komunikovat s Ollama serverem přes API, zpracovávat odpovědi a optimalizovat výkon modelů. Tato dovednost je nezbytná pro vývoj aplikací, které vyžadují rychlé a bezpečné zpracování textu ve vlastním prostředí – například v rámci interních systémů, bezpečnostních řešení nebo edge computing aplikací.

Integrace Ollama API je v praxi velmi užitečná například pro vývoj chatbotů, analýzu textu, automatizaci úloh a vytváření custom AI řešení. Student po této kapitole získá schopnost vytvářet robustní Pythonové klienty, které mohou komunikovat s lokálním Ollama serverem a efektivně zpracovávat odpovědi modelů.

---

## 2. Hlavní koncepty

### **Ollama REST API endpoints**

Ollama poskytuje řadu REST endpointů, které umožňují komunikaci s běžícím serverem. Nejčastěji používané jsou:

- `/api/generate` – generuje text na základě vstupního promptu.
- `/api/chat` – umožňuje interaktivní konverzaci (chatování) s modelem.
- `/api/tags` – získá seznam dostupných modelů.
- `/api/ps` – získá informace o běžících procesech.
- `/api/embeddings` – vytváří embeddingy z textu.

Například volání generovacího endpointu může vypadat takto:

```http
POST /api/generate HTTP/1.1
Host: localhost:11434
Content-Type: application/json

{
  "model": "llama3",
  "prompt": "Jak se dělá káva?",
  "stream": false
}
```

Odpověď se vrací ve formátu JSON:

```json
{
  "response": "Kávu se připravuje vařením vody a mlíčných produktů...",
  "done": true,
  "context": [123, 456, 789]
}
```

Tato struktura umožňuje jednoduchou integraci do jakékoli aplikace bez potřeby složitých knihoven.

---

### **Python client implementation**

Implementace Python klienta pro Ollama API je poměrně jednoduchá díky použití `requests` knihovny. Následující kód ukazuje jednoduché volání:

```python
import requests

def generate_text(prompt, model="llama3"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    response = requests.post(url, json=payload)
    return response.json()["response"]
```

Tento základní klient může být rozšířen o logiku pro streamování, kontroly chyb nebo přidání parametrů jako `temperature` a `max_tokens`.

---

### **Streaming responses**

Při použití streamování v Ollama API je možné získávat odpovědi postupně, což je velmi užitečné pro vizuální představu generace textu nebo pro reálné interaktivní aplikace. Nastavení `stream: true` v požadavku způsobí, že odpověď se posílá postupně v jednotlivých částech:

```python
def stream_response(prompt):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": "llama3",
        "prompt": prompt,
        "stream": True
    }
    response = requests.post(url, json=payload, stream=True)
    for line in response.iter_lines():
        if line:
            data = json.loads(line)
            print(data["response"], end="", flush=True)
```

Tato metoda umožňuje vytvářet plynulé uživatelské prostředí, například při simulaci chatu.

---

### **Model parameters tuning**

Při volání API můžeš upravovat parametry modelu pro optimalizaci výstupu. Klíčové parametry zahrnují:

- `temperature` – ovlivňuje náhodnost odpovědí (0 = deterministické, 1 = náhodné)
- `max_tokens` – maximální počet tokenů ve výstupu
- `top_p` – filtruje tokeny podle pravděpodobnosti
- `stop` – řetězce, které ukončí generování

Příklad:

```python
payload = {
    "model": "llama3",
    "prompt": "Vysvětli Python",
    "temperature": 0.7,
    "max_tokens": 200,
    "top_p": 0.9
}
```

Tato úprava umožňuje výrazně ovlivnit styl a délku odpovědi.

---

### **Context window management**

Každý model má omezenou kontextovou velikost (např. 4096 tokenů). Při použití chatovacího endpointu se kontext udržuje v rámci relace. Pokud je kontext příliš dlouhý, model může začít „zapomínat“ dřívější části konverzace.

Pro správné řízení kontextu je vhodné limitovat počet zpráv v historii a přidávat jejich souhrny. Například:

```python
messages = [
    {"role": "user", "content": "Jak se dělá káva?"},
    {"role": "assistant", "content": "Kávu se vaří z mléčných produktů..."},
]
```

Při větší konverzaci je vhodné přidat logiku pro shromažďování kontextu a jeho úpravu.

---

### **Token counting**

Tokenizace je klíčová pro správné nastavení parametrů a optimalizaci výkonu. Ollama používá tokenizer (např. `llama3` používá `LlamaTokenizer`). V Pythonu lze tokeny počítat pomocí knihovny `tokenizers`.

```python
from tokenizers import Tokenizer
from tokenizers.models import BPE

tokenizer = Tokenizer.from_file("path/to/tokenizer.json")
tokens = tokenizer.encode("Vítejte v Ollama API").ids
print(len(tokens))
```

Při práci s velkými texty je důležité monitorovat počet tokenů a omezoval je podle kapacity modelu.

---

### **Error handling strategies**

V praxi mohou nastat různé chyby, například:

- `ConnectionError` – server není dostupný
- `Timeout` – odpověď trvá příliš dlouho
- `HTTP 404` – model neexistuje
- `HTTP 500` – interní chyba serveru

Doporučené řešení:

```python
import requests
from requests.exceptions import RequestException

try:
    response = requests.post(url, json=payload, timeout=30)
    response.raise_for_status()
except RequestException as e:
    print(f"Chyba při komunikaci s Ollama: {e}")
```

Také je dobré implementovat opakování (retry) v případě dočasných chyb.

---

## 3. Důležité detaily

### **Časté chyby a jak se jim vyhnout**

- Není spuštěný Ollama server → zkontroluj, že `ollama serve` běží.
- Neplatný model → ověř existenci modelu přes `/api/tags`.
- Příliš dlouhý kontext → implementuj logiku pro zkracování nebo souhrny.
- Chybné parametry API → používej validaci a dokumentaci.

### **Best practices**

- Vždy testuj připojení k serveru před odesláním požadavku.
- Implementuj logiku pro streamování, pokud to podporuje model.
- Používej `timeout` v požadavcích, aby se aplikace neseděla.
- Udržuj výstupní formát konzistentní pro různé modely.

### **Performance tipy**

- Optimalizuj `max_tokens` podle potřeby – příliš dlouhé odpovědi snižují rychlost.
- Používej modely s menší velikostí, pokud je to možné.
- Udržuj kontext v rámci limitu modelu (např. 2048 tokenů pro některé modely).

---

## 4. Propojení s předchozími kapitolami

Tato kapitola navazuje na znalosti z předchozích kapitol, jako jsou instalace Ollama, práce s modely a základy REST API. Naučí studenty, jak přenést teoretické vědomosti do praxe – jak napojit aplikaci na model a efektivně komunikovat s ním.

Tato kapitola rozšiřuje již naučené schopnosti vytvářet API klienty a pracovat s datovými strukturami, přičemž přidává konkrétní použití v kontextu lokálního AI modelu. Výsledkem je aplikace, která dokáže efektivně komunikovat s AI modely přímo z vlastního zařízení – což je klíčové pro bezpečné a efektivní řešení ve velkém měřítku.

--- 

**Konec kapitoly: Ollama API integrace**


## 💻 Praktické příklady

<div style="background: #e8f5e9; padding: 20px; border-left: 5px solid #4caf50; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">👨‍💻 Hands-on příklady ke spuštění</h3>
    <p>Následující příklady si můžete hned vyzkoušet. Každý příklad je samostatně spustitelný.</p>
</div>


# Příklady integrace Ollama API

## 1. Základní komunikace s modely

**Co příklad demonstruje**: Základní volání REST API Ollamy pro získání odpovědi od modelu.

```python
import requests
import json

# Název příkladu: Základní komunikace s modely
# Co příklad demonstruje: Základní volání REST API Ollamy pro získání odpovědi od modelu.

def basic_ollama_call():
    # URL endpointu Ollamy (předpokládáme, že běží na localhostu)
    url = "http://localhost:11434/api/generate"
    
    # Parametry pro volání modelu
    payload = {
        "model": "llama3",  # Použijeme model Llama3
        "prompt": "Jak se dělá káva?",
        "stream": False     # Nechceme streamování
    }
    
    try:
        # Odeslání HTTP POST požadavku
        response = requests.post(url, json=payload)
        
        # Kontrola stavu odpovědi
        if response.status_code == 200:
            # Získání JSON odpovědi
            data = response.json()
            
            # Výpis odpovědi modelu
            print("Odpověď modelu:")
            print(data["response"])
            
            return data["response"]
        else:
            print(f"Chyba při volání API: {response.status_code}")
            return None
            
    except requests.exceptions.RequestException as e:
        print(f"Chyba při komunikaci s API: {e}")
        return None

# Spuštění funkce
if __name__ == "__main__":
    basic_ollama_call()
```

**Očekávaný výstup**:
```
Odpověď modelu:
Kávu se dělá různými způsoby, ale nejběžnější metoda je použití kávovaru nebo filtru. 
Nejdříve se zrnka kávy mlí, poté se připraví voda a nakonec se vše promíchá.
```

**Vysvětlení fungování**: Tento kód demonstruje základní komunikaci s Ollama REST API. Volá endpoint `/api/generate` s parametrem modelu, dotazem a nastavením streamování na `False`. Po úspěšném volání se extrahuje odpověď z JSON dat.

---

## 2. Streamovaná odpověď

**Co příklad demonstruje**: Streamování odpovědí od modelu po jednotlivých tokenů.

```python
import requests
import json

# Název příkladu: Streamovaná odpověď
# Co příklad demonstruje: Streamování odpovědí od modelu po jednotlivých tokenů.

def streaming_response():
    # URL endpointu Ollamy
    url = "http://localhost:11434/api/generate"
    
    # Parametry pro streamování
    payload = {
        "model": "llama3",
        "prompt": "Vysvětli kryptoměny jednoduše.",
        "stream": True     # Zapneme streamování
    }
    
    try:
        # Odeslání požadavku s streamováním
        response = requests.post(url, json=payload, stream=True)
        
        if response.status_code == 200:
            print("Streamovaná odpověď:")
            
            # Čtení streamu po jednotlivých řádcích (JSON objektech)
            for line in response.iter_lines():
                if line:
                    # Každý řádek je samostatný JSON objekt
                    data = json.loads(line)
                    
                    # Pokud máme token, vypíšeme jej
                    if "response" in data and data["response"]:
                        print(data["response"], end="", flush=True)
                        
            print()  # Nový řádek na konci
        else:
            print(f"Chyba při volání API: {response.status_code}")
            
    except requests.exceptions.RequestException as e:
        print(f"Chyba při komunikaci s API: {e}")

# Spuštění funkce
if __name__ == "__main__":
    streaming_response()
```

**Očekávaný výstup**:
```
Streamovaná odpověď:
Kryptoměny jsou digitální měny, které fungují na technologii blockchain. 
Každá transakce je zaznamenána do bloku a tento systém zajišťuje bezpečnost a průhlednost.
```

**Vysvětlení fungování**: Tento příklad ukazuje, jak pracovat s streamovanými odpověďmi. Používáme `stream=True` a `iter_lines()` pro čtení odpovědi po jednotlivých řádcích. Každý řádek je samostatný JSON objekt, který obsahuje token odpovědi.

---

## 3. Nastavení parametrů modelu

**Co příklad demonstruje**: Nastavování různých parametrů modelu (temperature, max_tokens) pro optimalizaci výstupu.

```python
import requests
import json

# Název příkladu: Nastavení parametrů modelu
# Co příklad demonstruje: Nastavování různých parametrů modelu pro optimalizaci výstupu.

def set_model_parameters():
    # URL endpointu Ollamy
    url = "http://localhost:11434/api/generate"
    
    # Parametry modelu s nastavením různých parametrů
    payload = {
        "model": "llama3",
        "prompt": "Napiš krátký příběh o objeviteli.",
        "options": {
            "temperature": 0.7,      # Náhodnost odpovědi (0-1)
            "max_tokens": 200,       # Maximální počet tokenů výstupu
            "top_p": 0.9,            # Top-p sampling
            "repeat_penalty": 1.1    # Penalty za opakování tokenů
        },
        "stream": False
    }
    
    try:
        response = requests.post(url, json=payload)
        
        if response.status_code == 200:
            data = response.json()
            print("Odpověď modelu s nastavenými parametry:")
            print(data["response"])
            
            # Vypsání použitých parametrů
            print("\nPoužité parametry:")
            for key, value in payload["options"].items():
                print(f"  {key}: {value}")
                
        else:
            print(f"Chyba při volání API: {response.status_code}")
            
    except requests.exceptions.RequestException as e:
        print(f"Chyba při komunikaci s API: {e}")

# Spuštění funkce
if __name__ == "__main__":
    set_model_parameters()
```

**Očekávaný výstup**:
```
Odpověď modelu s nastavenými parametry:
V době, kdy lidé ještě neuměli číst, objevil jeden chytrý muž způsob, jak se spolu komunikovat. 
Pomocí písmen a slov mohl přenášet své myšlenky a vědomosti.

Použité parametry:
  temperature: 0.7
  max_tokens: 200
  top_p: 0.9
  repeat_penalty: 1.1
```

**Vysvětlení fungování**: Tento příklad ukazuje, jak nastavit různé parametry modelu pomocí sekce `options` v JSON payloadu. Parametry jako `temperature` ovlivňují náhodnost odpovědi, `max_tokens` omezuje délku výstupu a `top_p` ovlivňuje distribuci pravděpodobnosti tokenů.

---

## 4. Správa kontextu (Context window)

**Co příklad demonstruje**: Správa kontextu mezi více voláními modelu pro vytváření rozsáhlejších odpovědí.

```python
import requests
import json

# Název příkladu: Správa kontextu (Context window)
# Co příklad demonstruje: Správa kontextu mezi více voláními modelu pro vytváření rozsáhlejších odpovědí.

def context_management():
    url = "http://localhost:11434/api/generate"
    
    # První dotaz - založení kontextu
    first_prompt = "Vysvětli, co je umělá inteligence."
    
    payload1 = {
        "model": "llama3",
        "prompt": first_prompt,
        "stream": False
    }
    
    response1 = requests.post(url, json=payload1)
    
    if response1.status_code == 200:
        data1 = response1.json()
        print("První odpověď:")
        print(data1["response"])
        
        # Druhý dotaz - využití kontextu
        second_prompt = "A jaké jsou hlavní typy AI?"
        
        payload2 = {
            "model": "llama3",
            "prompt": first_prompt + "\n" + data1["response"] + "\n" + second_prompt,
            "stream": False
        }
        
        response2 = requests.post(url, json=payload2)
        
        if response2.status_code == 200:
            data2 = response2.json()
            print("\nDruhá odpověď (využití kontextu):")
            print(data2["response"])
            
    else:
        print("Chyba při volání API")

# Spuštění funkce
if __name__ == "__main__":
    context_management()
```

**Očekávaný výstup**:
```
První odpověď:
Umělá inteligence je technologie, která simuluje lidské myšlení a schopnosti. 
Může se uč

## 🎯 Cvičení a úkoly

<div style="background: #fff3e0; padding: 20px; border-left: 5px solid #ff9800; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">✍️ Praktická cvičení k procvičení</h3>
    <p>Vyřešte následující úkoly. Začněte od jednoduššších a postupujte k složitějším.</p>
</div>

# Cvičení: Ollama API integrace

---

## 1. **Název úkolu**
Vytvoření jednoduchého chatovacího klienta pro Ollama API

### **Detailní zadání**
Vytvořte Python aplikaci, která komunikuje s Ollama API prostřednictvím HTTP požadavků. Aplikace by měla umožnit uživateli zadávat zprávy a dostávat odpovědi od LLM modelu (např. `llama3`). Použijte POST požadavek k odeslání dotazu do Ollama API a extrahujte odpověď ze získaného JSON objektu.

### **Vstupní data/požadavky**
- Model: `llama3`
- Textový vstup od uživatele (např. "Jak se dělá káva?")
- Očekávané použití: Zpráva z konzole

### **Očekávaný výstup**
Aplikace vytiskne odpověď od modelu na obrazovku. Například:
```
Uživatel: Jak se dělá káva?
Odpověď: Kávu se dělá vařením vody a mlíčných nebo kávových zrn.

```

### **Hints/Nápověda**
1. Použijte knihovnu `requests` pro odeslání POST požadavku.
2. Vytvořte JSON tělo s klíči `model` a `prompt`.
3. Zpracujte odpověď z API – pozor na `response.json()['message']['content']`.
4. Implementujte jednoduchou smyčku pro pokračování v konverzaci.

### **Kostra řešení**
```python
import requests

def chat_with_ollama(prompt):
    url = "http://localhost:11434/api/generate"
    data = {
        "model": "llama3",
        "prompt": prompt
    }
    # Odešlete požadavek a zpracujte odpověď

if __name__ == "__main__":
    while True:
        user_input = input("Uživatel: ")
        if user_input.lower() in ["quit", "exit"]:
            break
        response = chat_with_ollama(user_input)
        print(f"Odpověď: {response}")
```

### **Bonusové rozšíření**
- Přidejte historii konverzace (kontext) do požadavku.
- Umožněte výběr modelu přes argumenty.

---

## 2. **Název úkolu**
Integrace Ollama API s vlastním streamovaným generátorem odpovědí

### **Detailní zadání**
Vytvořte Python skript, který používá streamované odpovědi z Ollama API. Při zadání dotazu by měla aplikace postupně vypisovat znaky odpovědi (streaming), než se celá odpověď dokončí.

### **Vstupní data/požadavky**
- Model: `llama3`
- Zpráva od uživatele (např. "Napiš krátký text o přírodě")

### **Očekávaný výstup**
Postupné vypsání odpovědi znak po znaku nebo řádek po řádku, podobně jako chatovací rozhraní.

### **Hints/Nápověda**
1. Použijte `stream=True` ve volání `requests.post()`.
2. Iterujte přes `response.iter_lines()` pro zpracování streamu.
3. Zpracujte JSON řádky (`line.decode("utf-8")`) a extrahujte text.
4. Umožněte uživateli vypsat odpověď po jednotlivých částech.

### **Kostra řešení**
```python
import requests

def stream_response(prompt):
    url = "http://localhost:11434/api/generate"
    data = {
        "model": "llama3",
        "prompt": prompt,
        "stream": True
    }
    response = requests.post(url, json=data, stream=True)
    # Zpracuj stream a tiskni výstup

if __name__ == "__main__":
    user_input = input("Dotaz: ")
    stream_response(user_input)
```

### **Bonusové rozšíření**
- Přidejte možnost přerušení streamu (např. klávesa `q`).
- Umožněte zápis streamované odpovědi do souboru.

---

## 3. **Název úkolu**
Implementace API proxy serveru pro Ollama

### **Detailní zadání**
Vytvořte jednoduchý Flask server, který předává požadavky na Ollama API a zpět od něj. Tento server by měl fungovat jako proxy, tedy klient se obrací na váš server, ten pak volá Ollama a odpovídá zpět.

### **Vstupní data/požadavky**
- HTTP endpoint `/chat`
- Metoda: POST
- Vstup: JSON s klíči `prompt` a `model`

### **Očekávaný výstup**
Server přijímá požadavek, předává ho do Ollama a vrátí odpověď klientovi.

### **Hints/Nápověda**
1. Použijte Flask pro tvorbu API endpointu.
2. Vytvořte funkci, která komunikuje s `http://localhost:11434/api/generate`.
3. Předávejte JSON request přímo do Ollama.
4. Vrátíte odpověď jako JSON.

### **Kostra řešení**
```python
from flask import Flask, request, jsonify
import requests

app = Flask(__name__)

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    ollama_url = "http://localhost:11434/api/generate"
    response = requests.post(ollama_url, json=data)
    return jsonify(response.json())

if __name__ == '__main__':
    app.run(port=5000)
```

### **Bonusové rozšíření**
- Přidejte middleware pro logování požadavků.
- Ošetřete chyby z Ollama API (např. při špatném modelu).

---

## 4. **Název úkolu**
Vytvoření nástroje pro správu modelů v Ollama

### **Detailní zadání**
Napište skript, který umožňuje spravovat modely pomocí Ollama API (např. stahovat modely, seznam modelů, mazat jejich lokální kopie). Využijte GET/DELETE endpointy Ollama API.

### **Vstupní data/požadavky**
- Možnosti: `list`, `pull`, `delete`
- Parametry jako název modelu (`llama3`, `mistral`)

### **Očekávaný výstup**
Skript podporuje příkazovou řádku s podporkami pro jednotlivé operace (např. `python manager.py pull llama3`).

### **Hints/Nápověda**
1. Použijte `argparse` pro zpracování argumentů.
2. Využijte GET `/api/tags` pro seznam modelů.
3. POST na `/api/generate` pro stahování modelu (`pull`).
4. DELETE na `/api/generate` pro smazání.

### **Kostra řešení**
```python
import requests
import argparse

def list_models():
    response = requests.get("http://localhost:11434/api/tags")
    print(response.json())

def pull_model(model):
    data = {"name": model}
    response = requests.post("http://localhost:11434/api/generate", json=data)
    print(f"Model {model} downloaded")




## 📚 Další zdroje a materiály

<div style="background: #e3f2fd; padding: 20px; border-left: 5px solid #2196f3; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🔗 Doporučené materiály k dalšímu studiu</h3>
</div>

# Zdroje pro kapitolu: Ollama API integrace

## 1. Doporučené články a tutoriály

1. **"Integrating Ollama API with Python Applications"** – *Real Python*  
   Popis: Článek se zaměřuje na praktické použití Ollama API v Pythonu, včetně připojení k modelům, odesílání dotazů a zpracování odpovědí. Ideální pro začátečníky v programování.

2. **"Getting Started with Ollama and LLMs"** – *Hugging Face Blog*  
   Popis: Výukový článek, který vysvětluje, jak spustit Ollama na lokálním počítači a jak integrovat jej s různými LLM (Large Language Models). Obsahuje i ukázky kódu pro běžné scénáře.

3. **"Building Chatbots with Ollama API and Node.js"** – *FreeCodeCamp*  
   Popis: Tutoriál pro vývojáře, kteří chtějí vytvářet chatboty pomocí Ollama API a Node.js. Vysvětluje základní principy API volání, streaming odpovědí a integraci do webové aplikace.

4. **"Using Ollama in Production Environments"** – *Medium Article by AI Engineer*  
   Popis: Článek se zaměřuje na praxe pro nasazení Ollama API v produkčních prostředích, zahrnuje optimalizace, zpracování chyb a správu tokenů.

5. **"Ollama vs Local LLMs: A Practical Guide"** – *AI Weekly*  
   Popis: Praktický průvodce srovnáním Ollama API s jinými nástroji pro lokální spuštění LLM modelů. Užitečné pro vývojáře, kteří hledají nejlepší řešení pro jejich projekt.

## 2. YouTube videa

1. **"Ollama API Tutorial - Build a Chatbot with Python"** – *15 minut*  
   Popis: Krok za krokem tutoriál, jak vytvořit chatbot pomocí Ollama API a Pythonu. Videa obsahuje i ladění problémů s připojením a zpracováním odpovědí.

2. **"How to Use Ollama in Your Web App - Full Demo"** – *YouTube: Web Dev Simplified* – *20 minut*  
   Popis: Ukázkové využití Ollama API ve webové aplikaci s využitím JavaScriptu. Demonstruje integraci do Reactu a backendu pomocí Express.js.

3. **"Ollama API Deep Dive - From Zero to Production"** – *Tech With Tim* – *25 minut*  
   Popis: Komplexní výklad Ollama API s praktickými příklady, zahrnuje integraci do aplikací a základy správy modelů i deploymentu do cloudu.

## 3. Knihy a oficiální dokumentace

1. **Ollama Documentation** – [https://ollama.com/docs](https://ollama.com/docs)  
   Popis: Oficiální dokumentace Ollama, která poskytuje kompletní informace o instalaci, nastavení a rozhraní API. Vhodná pro každého vývojáře.

2. **"Building AI Applications with LLMs"** – *by Daniel Bourke*  
   Popis: Knihu doporučujeme jako další zdroj pro pochopení práce s LLM a jejich integrací do aplikací, včetně praktických příkladů s Ollama API.

3. **"Hands-On Machine Learning with Python"** – *Aurélien Géron*  
   Popis: Příručka zaměřená na praxi


## 📝 Shrnutí kapitoly

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 10px; color: white; margin: 30px 0;">
    <h3 style="color: white; margin-top: 0;">✅ Co jste se naučili</h3>
    <ul style="list-style: none; padding-left: 0;">
        <li>✓ Ollama REST API endpoints</li>
<li>✓ Python client implementation</li>
<li>✓ Streaming responses</li>
<li>✓ Model parameters tuning</li>
<li>✓ Context window management</li>
    </ul>
    
    <h3 style="color: white;">🎯 Klíčové dovednosti</h3>
    <p>Po dokončení této kapitoly byste měli být schopni prakticky použít všechny probrané koncepty.</p>
    
    <h3 style='color: white;'>➡️ Další kapitola</h3><p>Kapitola 33 - pokračujte ve studiu!</p>
</div>

---

*📅 Notebook vygenerován: 2025-09-29 13:17:34*  
*🤖 Generátor: Comprehensive Colab Generator v2.0*  
*📚 Učebnice programování - Od základů k AI*


## 🧪 Sandbox - Prostor pro experimenty

Použijte následující buňky pro vlastní experimenty a testování:

In [ ]:
# 🧪 Zde můžete experimentovat s kódem z kapitoly
# Napište svůj kód zde:

